In [1]:
import numpy as np
import matplotlib.pyplot as plt
import json 

In [91]:
dataset_dir = '/mnt/c/Users/dev/Documents/fresco/dataset'
alignemnt_filename = '/mnt/c/Users/dev/Documents/fresco/fresco-reassembling/code/alignments/alignments2.json'

In [92]:
transforms = json.load(open(dataset_dir + '/transforms.json'))
# transforms

In [93]:
alignments = json.load(open(alignemnt_filename))

In [94]:
class Transform:
    def __init__(self, x, y, angle):
        self.x = x
        self.y = y
        self.angle = angle
        
    def __eq__(self, other):
        return (self.x - other.x)**2 + (self.y - other.y)**2 < 15**2 and (abs(self.angle - other.angle) < 10 or abs(self.angle - other.angle) > 350)

    def __repr__(self):
        return f'Transform({self.x}, {self.y}, {self.angle})'
    def __str__(self):
        return f'Transform({self.x}, {self.y}, {self.angle})'

In [95]:
tr1 = Transform(0, 0, -178)
tr2 = Transform(5, 2, 178)
print(tr1 == tr2)

True


In [96]:
pred_transforms = {}
gt_transforms = {}

In [97]:
for pair in alignments.keys():
    l, r = pair.split('_')
    l, r = int(l), int(r)
    pair_aligns = alignments[pair]
    pair_transforms = [Transform(pair_align['x'], pair_align['y'], pair_align['angle']) for pair_align in pair_aligns]
    pred_transforms[(l, r)] = pair_transforms

In [98]:
for pair in transforms.keys():
    l, r = pair.split('_')
    l, r = int(l), int(r)
    pair_align = transforms[pair]
    pair_transforms = [Transform(pair_align[0], pair_align[1], pair_align[2])]
    gt_transforms[(l, r)] = pair_transforms

In [99]:
pred_transforms[(6, 7)], gt_transforms[(6, 7)]

([Transform(-67, -67, 4.77326251632438)], [Transform(-63.0, -62.5, 0)])

In [100]:
print(pred_transforms[(6, 7)][0] == gt_transforms[(6, 7)][0])
print(gt_transforms[(6, 7)][0] in pred_transforms[(6, 7)])

True
True


In [101]:
set([pair[0] for pair in pred_transforms.keys()] + [pair[1] for pair in pred_transforms.keys()]) 

{3, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 49, 53, 54, 59, 61, 65, 95, 96}

In [102]:
def compute_recall(pred_transforms, gt_transforms, frag_nums=None):
    if frag_nums is None:
        frag_nums = list(set([pair[0] for pair in pred_transforms.keys()] + [pair[1] for pair in pred_transforms.keys()]))
    frag_nums.sort()
    
    tp = 0
    fn = 0
    for l in frag_nums:
        for r in frag_nums:
            if l >= r:
                continue
            if (l, r) not in gt_transforms:
                continue
            gt = gt_transforms[(l, r)][0]
            if gt in pred_transforms[(l, r)]:
                print(f'pair ({l}, {r}) correct')
                tp += 1
            else:
                print(f'pair ({l}, {r}) incorrect')
                fn += 1
    return tp / (tp + fn)

In [103]:
compute_recall(pred_transforms, gt_transforms, frag_nums=[3, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 49, 53, 54, 59, 61, 65, 95, 96])

pair (3, 5) incorrect
pair (5, 7) incorrect
pair (5, 9) correct
pair (6, 7) correct
pair (6, 9) correct
pair (7, 8) correct
pair (8, 59) correct
pair (9, 11) correct
pair (9, 15) correct
pair (9, 59) incorrect
pair (9, 61) correct
pair (10, 11) correct
pair (10, 12) incorrect
pair (11, 12) incorrect
pair (11, 14) incorrect
pair (14, 15) incorrect
pair (15, 61) correct
pair (49, 53) correct
pair (49, 54) correct
pair (53, 54) correct
pair (53, 61) correct
pair (54, 61) incorrect
pair (54, 65) incorrect
pair (59, 61) correct
pair (59, 95) correct
pair (61, 96) incorrect
pair (65, 96) correct
pair (95, 96) correct


0.6428571428571429